In [ ]:
import mysql.connector

# establish connection
cnx = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="littlelemondb"
)

# create a cursor
cursor = cnx.cursor()

# GetMaxQuantity
cursor.execute("""
    CREATE PROCEDURE GetMaxQuantity ()
    BEGIN
        SELECT MAX(OrderQty)
        FROM `order`;
    END""")
cursor.callproc('GetMaxQuantity')
for result in cursor.stored_results():
    print(result.fetchall())

# ManageBooking
cursor.execute("""
    CREATE PROCEDURE ManageBooking (
        IN booking_date DATE,
        IN table_number INT
    )
    BEGIN
        DECLARE booking_status BOOL;
        SELECT COUNT(*) INTO booking_status
        FROM booking
        WHERE booking_date = booking_date AND table_number = table_number;
        IF booking_status = TRUE THEN
            SELECT CONCAT('Table ', table_number, ' is already booked') AS message;
        ELSE
            SELECT CONCAT('Table ', table_number, ' is available') AS message;
        END IF;
    END""")
cursor.callproc('ManageBooking', ('2023-03-15', 1))
for result in cursor.stored_results():
    print(result.fetchall())

# UpdateBooking
cursor.execute("""
    CREATE PROCEDURE UpdateBooking(
        IN booking_id INT,
        IN booking_table INT,
        IN booking_date DATE
    )
    BEGIN
        UPDATE booking 
        SET BookingTable = booking_table, BookingDate = booking_date 
        WHERE BookingID = booking_id;
    END""")
cursor.callproc('UpdateBooking', (1, 1, '2022-10-11'))

# AddBooking
cursor.execute("""
    CREATE PROCEDURE AddBooking (
        IN table_number INT,
        IN booking_date DATE
    )
    BEGIN
        DECLARE booking_status BOOL;
        SELECT COUNT(*) INTO booking_status
        FROM booking
        WHERE BookingTable = table_number AND BookingDate = booking_date;
        IF booking_status = 0 THEN
            INSERT INTO booking (BookingTable, BookingDate)
            VALUES (table_number, booking_date);
            SELECT CONCAT('Booking for table ', table_number, ' on ', booking_date, ' was successful') AS message;
        ELSE
            SELECT CONCAT('Table ', table_number, ' is already booked on ', booking_date) AS message;
        END IF;
    END""")
cursor.callproc('AddBooking', (1, '2023-03-15'))
for result in cursor.stored_results():
    print(result.fetchall())

# CancelBooking
cursor.execute("""
    CREATE PROCEDURE CancelBooking(IN booking_id INT)
    BEGIN
        DELETE FROM booking WHERE BookingID = booking_id;
    END""")
cursor.callproc('CancelBooking', (1,))

# close the cursor and connection
cursor.close()
cnx.close()
